In [ ]:
from collections import Counter
import datetime
import pandas as pd
import numpy as np
from pyspark import SparkContext, SparkConf
!pip install biopython
import pandas as pd
import pandas as pd
from collections import Counter
import numpy as np
from Bio import pairwise2
from Bio.pairwise2 import format_alignment


path='TESI/SPIKE FASTA/spikeprot0329.fasta'
day_list=[]
month_list=[]
year_list=[]
date_list=[]
epi_list=[]
hosp_list=[]
leng=[]
seq_list=[]
flag=False
with open(path) as f:
    for line in f.readlines():
        if flag==True:
            seq_list.append(line.strip())
            flag=False
        if 'England' in line:
            features=(str(line)).split('|')
            if features[6] == 'Human':
                name=features[1].split('/')[1]
                year,month,day=features[2].split('-')
                if month == '00':
                    continue
                if day== '00':
                    continue
                cos= year + '-'+month +'-'+ day
                data= datetime.datetime.strptime(cos, '%Y-%m-%d')
                date_list.append(data)
                day_list.append(int(day))
                month_list.append(int(month))
                year_list.append(int(year))
                epi=features[3].split('_')[2]
                epi_list.append(epi)
                lab=features[7].split('/')[0]
                if lab=='Originating lab':
                    lab=features[8]
                hosp_list.append(lab)
                flag=True
            
                

df=pd.DataFrame()

df['EPI ISL']=epi_list
df['ORIGIN LAB']=hosp_list
df['DAY']=day_list
df['MONTH']=month_list
df['YEAR']=year_list
df['DATE']=date_list
df['Sequence']=seq_list

df=df.sort_values(by=['YEAR','MONTH','DAY'])
df=df.reset_index()
df=df.drop(columns=['YEAR','MONTH','DAY','index'])
#df.to_csv('dataframe.csv')

conf = SparkConf().setAppName('esempio_').setMaster('local[*]')
sc = SparkContext(conf=conf)

data=df


sequences=data['Sequence'].values

dicti=Counter(sequences)
ref_seq='MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQDVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPRRARSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQDVVNQNAQALNTLVKQLSSNFGAISSVLNDILSRLDKVEAEVQIDRLITGRLQSLQTYVTQQLIRAAEIRASANLAATKMSECVLGQSKRVDFCGKGYHLMSFPQSAPHGVVFLHVTYVPAQEKNFTTAPAICHDGKAHFPREGVFVSNGTHWFVTQRNFYEPQIITTDNTFVSGNCDVVIGIVNNTVYDPLQPELDSFKEELDKYFKNHTSPDVDLGDISGINASVVNIQKEIDRLNEVAKNLNESLIDLQELGKYEQYIKWPWYIWLGFIAGLIAIVMVTIMLCCMTSCCSCLKGCCSCGSCCKFDEDDSEPVLKGVKLHYT'
sequences=list(set(sequences))
input_=sc.parallelize(sequences)
a=input_.map(lambda line: (pairwise2.align.globalms(ref_seq, line, 1, -1, -3, -2))[0][1])
sequences_align=a.collect()
data['Aligned']=np.zeros(len(data))
                         
for i,el in enumerate(sequences):
    mask= (data['Sequence']==el)
    data.loc[mask,'Aligned']=sequences_align[i]
    
    
months=[]
for el in data['DATE']:
    month=int(str(el).split('-')[1])
    months.append(month)
data['Month']=months
mask=[True if str(x).split('-')[0] == '2021' else False for x in data['DATE'].values]
data.loc[mask,'Month']= np.copy(data.loc[mask,'Month'])+12


data['Sequence'] = data['Sequence'].str.replace('*','')
data['Aligned'] = data['Aligned'].str.replace('*','')
name=path.split('/')[2][:13] + '_England.csv'
data.to_csv(name)